In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.compose import make_column_transformer,make_column_selector
from sklearn.metrics import *
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet,Ridge
import xgboost as xgb
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import TransformedTargetRegressor
import xgboost as xgb
from sklearn.svm import SVR
from sklearn.feature_selection import RFE
from prettytable import PrettyTable
from prettytable import PrettyTable
import time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import  cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler


%matplotlib inline

In [ ]:
df = pd.read_csv('C:/Users/Lynda/Desktop/anticipez les besoins en consommation de batiment/data_filtre.csv')
data = df.copy()

In [ ]:
data.columns

In [ ]:
# Imputation des valeurs manquantes par la moyenne
data.fillna(data.mean(), inplace=True)

#data.dropna(inplace = True)

In [ ]:
y = np.log(data['SiteEnergyUse(kBtu)'])
X = data.drop(columns=['TotalGHGEmissions','SiteEnergyUse(kBtu)'], axis = 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
print(f' X_train: {X_train.shape} \n X_test :  {X_test.shape}')

In [ ]:

def plot_learning_curve(
    estimator,title,X,y,axes=None,ylim=None,cv=None,n_jobs=None,train_sizes=np.linspace(0.1, 1.0, 5),scoring = None):
    
    if axes is None:
        _, axes = plt.subplots(1, 2, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    if scoring != None :
        axes[0].set_ylabel("Score - " + str(scoring))
    else :
        axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = learning_curve(
        estimator,
        X,
        y,
        cv=cv,
        n_jobs=n_jobs,
        train_sizes=train_sizes,
        return_times=True,
        scoring = scoring
    )

    plt.figure(figsize=(7,5))
    #plt.grid()
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(
        train_sizes,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.1,
        color="r",
    )
    axes[0].fill_between(
        train_sizes,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
        color="g",
    )
    axes[0].plot(
        train_sizes, train_scores_mean, "o-", color="r", label="Training score"
    )
    axes[0].plot(
        train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score"
    )
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, "o-")
    axes[1].fill_between(
        train_sizes,
        fit_times_mean - fit_times_std,
        fit_times_mean + fit_times_std,
        alpha=0.1,
    )
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")



    return plt.show()

In [ ]:
def model_cross_val_score(models, X, y, cv=5, scoring='r2'):
    t = PrettyTable(["Model",'Score', "std_acc (+/-)", 'Time'], reversesort = True)
    min_score = 0
    best_model = ''
    for reg in models: 
        st = time.time()
        scores = cross_val_score(reg[1], X, y, scoring =scoring , cv=cv, n_jobs = -1)
        # scoring : ' explained_variance_score', ' neg_mean_squared_error', 'r2', 'neg_mean_squared_error'
        t.add_row([reg[0],round(scores.mean(),4), round(scores.std(),4), round(time.time() - st,4)])
        if min_score < scores.mean():
            best_model = reg[1]
            name=reg[0]
            min_score = scores.mean()
    print (t)
    return (best_model,name,min_score)

In [ ]:
numerical_feautures = make_column_selector(dtype_include = np.number)
categorical_features = make_column_selector(dtype_exclude= np.number)
numerical_pipeline = make_pipeline(StandardScaler())
categorical_pipeline = make_pipeline(OneHotEncoder(handle_unknown = "ignore"))

preprocessor = make_column_transformer((numerical_pipeline,numerical_feautures),
                                        (categorical_pipeline,categorical_features))




In [ ]:
Ridg   = make_pipeline(preprocessor,Ridge())
linearregression = make_pipeline(preprocessor,LinearRegression())
elasticnet = make_pipeline(preprocessor ,  ElasticNet())
gradientboosting = make_pipeline(preprocessor, GradientBoostingRegressor())
randomforest = make_pipeline(preprocessor,RandomForestRegressor())  
svr  = make_pipeline(preprocessor,SVR())
xgboost = make_pipeline(preprocessor,xgb.XGBRegressor()) 

dict_of_models = { 'ridg': Ridg,
                    'linearregression':linearregression ,
                    'elasticnet': elasticnet,
                    'xgboost':xgboost,
                    'gradientboosting':gradientboosting,
                    'randomforest':randomforest,
                    'svr':svr ,
                    
}

In [ ]:
Ridg.get_params().keys()

In [ ]:
ridg_grid = { 
    'ridge__fit_intercept':[True,False],
    'ridge__alpha':np.logspace(-4,6,20)
}

linear_grid = {
'linearregression__fit_intercept':[True,False]
#'linearregression__normalize':[True,False]
}

elasticnet_grid = {
    "elasticnet__tol" : [0.1,0.01,0.001,0.0001],
    "elasticnet__alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],  #alpha, coef qui multiplie le terme de pénalité)
    "elasticnet__l1_ratio": np.arange(-1.0, 1.0, 0.2),
    'elasticnet__fit_intercept':[True,False]  
}

gb_grid = {
    'gradientboostingregressor__learning_rate':[0.001, 0.01, 0.1, 0.2, 0, 3],
    'gradientboostingregressor__max_depth' : [1, 6, 12, 20]
}

randomforest_grid = {
    'randomforestregressor__n_estimators' : [700, 900],
    #'randomforestregressor__max_depth': [1000,1400,1600 ],
   # 'randomforestregressor__min_samples_split': [10, 20], 
    'randomforestregressor__max_features': [4,10,30]
}

svr_grid = {
    'svr__gamma':[0.001, 0.01, 1],
    'svr__epsilon':[0, 0.01, 0.1, 0.5, 1],
    'svr__C':np.logspace(5, 0, 10)
}

xgboost_grid = {
    'xgbregressor__learning_rate': np.logspace(-2,0,10),
    'xgbregressor__gamma':[0, 0.25, 0.5, 1.0],
    # 'xgbregressor__max_depth':[1, 4, 10, 20],
    'xgbregressor__n_estimators':[50, 150, 200]
}



In [ ]:
grid_searchridge= GridSearchCV(Ridg , param_grid = ridg_grid  , n_jobs = -1, cv =5,scoring = 'r2')

grid_searchrelasticnet= GridSearchCV(elasticnet , param_grid = elasticnet_grid, n_jobs = -1, cv =5,scoring = 'r2')

grid_searchlinear= GridSearchCV(linearregression , param_grid = linear_grid , n_jobs = -1, cv =5,scoring = 'r2')

grid_searchxgboost= GridSearchCV(xgboost , param_grid = xgboost_grid, n_jobs = -1, cv =5,scoring = 'r2')

grid_searchrandomforest= GridSearchCV(randomforest , param_grid = randomforest_grid, n_jobs = -1, cv =5,scoring = 'r2')

grid_searchsvr= GridSearchCV(svr , param_grid = svr_grid, n_jobs = -1, cv =5,scoring = 'r2')

grid_searchgbt= GridSearchCV(gradientboosting , param_grid = gb_grid, n_jobs = -1, cv =5,scoring = 'r2')



In [ ]:
import sklearn
def evaluate(grid, X_train,y_train,X_test,y_test,name):
    
        score = 'explained_variance'
        start_time = time.time()
        result = grid.fit(X_train,y_train)
        training_time = time.time() - start_time

        print("="*30)
        print(name)
        print('\nBest Score to optimize: '+score+' = %s' % result.best_score_)
        print('Best Hyperparameters: %s' % result.best_params_)

        print('\n****Results****')
        predictions = result.predict(X_test)

        explained_variance = result.best_score_
        r2 = sklearn.metrics.r2_score(y_test, predictions)

        #print("Root Mean Squared Error: {}".format(rmse))
        #print("Mean Absolute Error: {}".format(mae))
        print("Explained Variance: {}".format(explained_variance))
        print("R2 Score: {}".format(r2))
        print("Training time: {}s".format(round(training_time, 2)))
        
        # features_names = result.feature_names_in_
        # #print(features_names)
        # features_importance = grid.best_estimator_[1].feature_importances_
        # order_importance = np.argsort(grid.best_estimator_[1].feature_importances_)[::-1]
        #print(order_importance)

        # GRAPHS
        fig = plt.figure(figsize=(8, 5))
        
        # plot valeurs réelles / valeurs prédites
        X_plot = [y_test.min(), y_test.max()]
        #plt.subplot(1,2,1)
        plt.scatter(y_test, predictions, alpha=.6)
        plt.plot(X_plot, X_plot, color='r')
        plt.title("Valeurs prédites VS valeurs réelles")
        plt.xlabel("Valeurs réelles")
        plt.ylabel("Valeurs prédites")

        # # Features importance

        if grid in [grid_searchrandomforest,grid_searchgbt,grid_searchxgboost]:
                feat = list(zip(result.feature_names_in_, grid.best_estimator_[1].feature_importances_))
                df_importances = pd.DataFrame(feat, columns=['Feature', 'Importance']).sort_values(by='Importance', ascending=False)
                df_importances.plot.barh(x='Feature', y='Importance')

          
        
        if grid in [grid_searchlinear,grid_searchridge,grid_searchrelasticnet, grid_searchsvr ]:

                features = X_train.columns
                feature_names = features.tolist()
                model = result.best_estimator_
                feature_importance = np.abs(model.steps[-1][1].coef_)
                pos = np.argsort(feature_importance)
                #print(feature_importance)
                reg_feat_importance = pd.DataFrame(columns=["Feature Name", "Feature Importance"])
                feat = list(zip(result.feature_names_in_, pos))
                df_importances = pd.DataFrame(feat, columns=['Feature', 'Importance']).sort_values(by='Importance', ascending=False)
                df_importances.plot.barh(x='Feature', y='Importance')
                

In [ ]:
# #feature_importance = clf.feature_importances_
# #grid_searchrandomforest.feature_names
# clf.feature_names_

# features_names = clf.get_feature_names_out()


## Modèles lineaires:

### Régression_linear:

In [ ]:
evaluate(grid_searchlinear,X_train,y_train,X_test,y_test,'Model_LinearRegressor')

In [ ]:
title = 'Linear_regression'
fig, axes = plt.subplots(1, 2, figsize=(20, 4))
plot_learning_curve(linearregression, title, X, y, axes=axes, cv=5)




#### Ridge:

In [ ]:
evaluate(grid_searchridge,X_train,y_train,X_test,y_test,'Model_Ridge')

### Learning curve:

In [ ]:
title = 'Modele_Ridge'
fig, axes = plt.subplots(1, 2, figsize=(20, 4))
plot_learning_curve(Ridg, title, X, y, axes=axes, cv=5)




### ElasticNet:

In [ ]:
evaluate(grid_searchrelasticnet,X_train,y_train,X_test,y_test,'Model_ElasticNet')

In [ ]:
title = 'Modele_Elasticnet'
fig, axes = plt.subplots(1, 2, figsize=(20, 4))
plot_learning_curve(elasticnet, title, X, y, axes=axes, cv=5)

### svr:

In [ ]:
evaluate(grid_searchsvr,X_train,y_train,X_test,y_test,'svr')

In [ ]:
title = 'svr'
fig, axes = plt.subplots(1, 2, figsize=(20, 4))
plot_learning_curve(svr, title, X, y, axes=axes, cv=5)

### Randomforestregressor:

In [ ]:
evaluate(grid_searchrandomforest,X_train,y_train,X_test,y_test,'RandomForestRogressor')

In [ ]:
title = 'RandomForestRegressor'
fig, axes = plt.subplots(1, 2, figsize=(20, 4))
plot_learning_curve(randomforest, title, X, y, axes=axes, cv=5)

### gradient bossting:

In [ ]:
evaluate(grid_searchgbt,X_train,y_train,X_test,y_test,'Gradientboosting')

In [ ]:
title = 'Gradient boosting'
fig, axes = plt.subplots(1, 2, figsize=(20, 4))
plot_learning_curve(gradientboosting, title, X, y, axes=axes, cv=5)

### XGboost:

In [ ]:
evaluate(grid_searchxgboost,X_train,y_train,X_test,y_test,'xgboost')

In [ ]:
title = 'xgboost'
fig, axes = plt.subplots(1, 2, figsize=(20, 4))
plot_learning_curve(xgboost, title, X, y, axes=axes, cv=5)

#### Selection du meilleur modèle:

In [ ]:
def run_model(X,y):
    models = [#('Ridg', grid_searchridge.fit(X,y)),

           # ('elasticnet', grid_searchridge.fit(X,y)),

           # ('linearregression' , grid_searchlinear.fit(X,y)),

             ('xgboost', grid_searchxgboost.fit(X,y)),

            #('svr', grid_searchsvr.fit(X,y).best_estimator_),

             ('randomforest' , grid_searchrandomforest.fit(X,y).best_estimator_),
             
            ('gradientboosting' ,grid_searchgbt.fit(X,y).best_estimator_)
    ]


    best_model = model_cross_val_score(models,X,y,cv = 5)
    return best_model
  
best_model_1=run_model(X_train, y_train)
   

* Sur les 4 modèles testés, les modèles linéaires retournent de moins bonnes métriques en général. Si nous prenons en considération le score Explained Variance, qui aura du sens sur les modèles linéaires et non-linéaires, l'algorithme xgboost offrent les meilleures performances la qualité des prédictions mais le temps de calcul est relativement plus long, randomforestregressor egalement offre de meilleurs performance meme en terme de temps.

On observe également que le passage au log améliore significativement les résultats des modèles non-linéaires. En effet, cela se justifie par certaines features étant très déviée proche de 0.

## Modélisation sans la variable ENERGYSTARScore

In [ ]:
y = np.log(data['SiteEnergyUse(kBtu)'])
X = data.drop(columns=['TotalGHGEmissions','SiteEnergyUse(kBtu)','ENERGYSTARScore'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
print(f' X_train: {X_train.shape} \n X_test :  {X_test.shape}')


* Evaluation avec le modele RandomForest:

In [ ]:
evaluate(grid_searchrandomforest,X_train,y_train,X_test,y_test,'RandomForestRogressor')

In [ ]:
title = 'RandomForestRegressor'
fig, axes = plt.subplots(1, 2, figsize=(20, 4))
plot_learning_curve(randomforest, title, X, y, axes=axes, cv=5)

*  évaluation avec le modèle xgboost:

In [ ]:
evaluate(grid_searchxgboost,X_train,y_train,X_test,y_test,'Xgboost')

In [ ]:
title = 'xgboost'
fig, axes = plt.subplots(1, 2, figsize=(20, 4))
plot_learning_curve(xgboost, title, X, y, axes=axes, cv=5)

In [ ]:
evaluate(grid_searchgbt,X_train,y_train,X_test,y_test,'Gradientboosting')

In [ ]:
def run_model(X,y):
    models = [

           

             ('xgboost', grid_searchxgboost.fit(X,y)),

             ('randomforest' , grid_searchrandomforest.fit(X,y).best_estimator_),
             
            ('gradientboosting' ,grid_searchgbt.fit(X,y).best_estimator_)
    ]


    best_model = model_cross_val_score(models,X,y,cv = 5)
    return best_model
  
best_model_1=run_model(X_train, y_train)
   

## Export des données prédites

In [ ]:
import sklearn
def predict(grid, X_train,y_train,X_test,y_test):
    
        score = 'explained_variance'
        start_time = time.time()
        result = grid.fit(X_train,y_train)
        training_time = time.time() - start_time

        print("="*30)
        
        print('\nBest Score to optimize: '+score+' = %s' % result.best_score_)
        print('Best Hyperparameters: %s' % result.best_params_)

        print('\n****Results****')
        predictions = result.predict(X_test)

        explained_variance = result.best_score_
        r2 = sklearn.metrics.r2_score(y_test, predictions)

        #print("Root Mean Squared Error: {}".format(rmse))
        #print("Mean Absolute Error: {}".format(mae))
        print("Explained Variance: {}".format(explained_variance))
        print("R2 Score: {}".format(r2))
        print("Training time: {}s".format(round(training_time, 2)))
        
        return np.exp(predictions)

In [ ]:
prediction = predict(grid_searchgbt, X_train.drop('ENERGYSTARScore', axis=1)
                       , y_train, X_test.drop('ENERGYSTARScore', axis=1)
                       , y_test)

In [ ]:
prediction

******
